# RAG (Retrieval-Augmented Generation)
Это мощная парадигма в AI и LLM. RAG совмещает извлечение информации с текстовой генерацией, чтобы улучшить производительность LLM добавляя внешние источники информации.

### Рассмотрим обычный флоу чат-бота без использования RAG:
<br>
<img src="1.png" width=200 height=200 />

Чат-бот может отвечать на общие вопросы, основываясь на тренировочном множестве, на котором его обучали, но не знать какой-то доменно-специфичной информации. Чтобы это преодолеть, мы должны предоставить модели доступ к внешней (недостающей) информации.

<br>
<img src="2.png" width=200 height=200 />

### Основные компоненты RAG системы

- **Retrieval model** - извлекает релевантную информацию из внешнего источника знаний, который может представлять собой базу данных, поисковый движок и т.д.
- **Language model** - генерирует ответ на основании извлеченных знаний.

### Простая RAG система

Создадим RAG-систему, которая извлекает информацию из предопределенного датасета и генерирует ответ на основании извлеченных знаний. Система будет состоять из следующих компонентов:

<br>
<img src="3.png" width=200 height=200 />

1. **Embedding model** - предобученная языковая модель, которая преобразует входной текст в эмбеддинги - векторные представления, которые улавливают семантику. 
2. **Vector database** - хранилище, которое содержит вектора (эмбеддинги).
3. **Chatbot** - языковая модель, которая генерирует ответ на основании извлеченной информации.

- Скачиваем ollama для локального запуска и использования моделей https://ollama.com/
- Скачиваем модели
```
ollama pull hf.co/CompendiumLabs/bge-base-en-v1.5-gguf
ollama pull hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF
```
- Устанавливаем ollama `pip install ollama`

### LLM без RAG

In [1]:
import ollama

EMBEDDING_MODEL = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
LANGUAGE_MODEL = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

input_query = input('Ask me a question: ')


stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': 'you are a helpfull assistant that answers all questions'},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Ask me a question:  tell me about cat speed


Chatbot response:
Cat speed! Also known as feline velocity, it's the fastest pace at which a domestic cat can run. Here are some fascinating facts and statistics about cat speed:

**How fast can cats run?**

On average, a healthy adult cat can run up to 25-30 miles per hour (mph) or 40-48 kilometers per hour (km/h). However, the fastest recorded cat speed is around 36 mph (58 km/h), achieved by a Bengal cat.

**Why are cats so fast?**

Cats' rapid running ability is due to their unique physiology. They have:

1. **Powerful leg muscles**: Cats have quadriceps muscles that allow them to generate high forces and accelerate quickly.
2. **Lightweight skeletal system**: Their bones are relatively lightweight, which makes it easier for them to move rapidly.
3. **Flexible joints**: Cats' joints are designed to be flexible, allowing them to stretch and change direction quickly.

**Other interesting facts about cat speed**

* Cats can reach speeds of up to 10-15 mph (16-24 km/h) over short dista

### Загружаем данные, на основе которых будет производиться поиск

In [2]:
dataset = []
with open('cat-facts.txt', 'r') as file:
    dataset = file.readlines()
    print(f'Loaded {len(dataset)} entries')

Loaded 150 entries


### Строим Векторное хранилище 
Каждый элемент векторного хранилища будет представлять собой кортеж `(chunk, embedding)` <br>
Эмбеддинг - это список float, к примеру: `[0.1, 0.04, -0.34, 0.21, ...]`

In [3]:
VECTOR_DB = []

def add_chunk_to_database(chunk):
  embedding = ollama.embed(model=EMBEDDING_MODEL, input=chunk)['embeddings'][0]
  VECTOR_DB.append((chunk, embedding))

In [4]:
for i, chunk in enumerate(dataset):
  add_chunk_to_database(chunk)
  

print('Added chunks')

Added chunks


In [5]:
VECTOR_DB[:10]

[('On average, cats spend 2/3 of every day sleeping. That means a nine-year-old cat has been awake for only three years of its life.\n',
  [-0.035897903,
   -0.023045903,
   0.04725552,
   -0.07919412,
   0.037888046,
   -0.014470911,
   0.07594431,
   0.053562913,
   -0.014032928,
   -0.002645408,
   -0.020073408,
   -0.007249306,
   -0.057428047,
   0.011650615,
   -0.046626985,
   0.041503306,
   0.08662891,
   0.009218618,
   -0.03105272,
   -0.029372955,
   0.003837944,
   0.026921792,
   -0.026493669,
   -0.0007908712,
   0.050383635,
   -0.015898,
   -0.010836049,
   -0.0014388829,
   0.0020308204,
   -0.013215007,
   0.03692541,
   -0.03137059,
   -0.034285862,
   0.007564012,
   0.026820643,
   -0.034794595,
   -0.013016162,
   -0.033246063,
   0.013256344,
   0.031907327,
   -0.033586092,
   -0.012728936,
   -0.018640054,
   0.013432499,
   -0.030404098,
   -0.016688813,
   -0.0017678647,
   -0.013123446,
   0.026660984,
   0.01537001,
   -0.032528613,
   0.0016620529,
   0.0

Для вычисления схожести между векторами можно использовать косинусное расстояние, евклидово расстояние или другие метрики. В данном случае будем использовать косинусное расстояние: <br>
$similarity(A,B)=cos(\theta)=\frac{A\cdot B}{||A|| ||B||} = \frac{\sum_{i=1}^{n}A_i B_i}{\sqrt{\sum_{i=1}^{n}A_i^2\sum_{i=1}^{n}B_i^2}}$

In [6]:
def cosine_similarity(a, b):
  dot_product = sum([x * y for x, y in zip(a, b)])
  norm_a = sum([x ** 2 for x in a]) ** 0.5
  norm_b = sum([x ** 2 for x in b]) ** 0.5
  return dot_product / (norm_a * norm_b)

### Извлечение 

- получаем входной запрос от пользователя
- вычисляем эмбеддинг для пользовательского запроса
- вычислем косинусное расстояние между эмбеддингом пользовательского запроса и эмбеддингами из векторного хранилища
- сортируем полученные расстояния по убыванию
- возвращаем top N расстояний и ответов
<br>
<img src="4.png" width=200 height=200 />


In [7]:
def retrieve(query, top_n=3):
  query_embedding = ollama.embed(model=EMBEDDING_MODEL, input=query)['embeddings'][0]
  similarities = []
  for chunk, embedding in VECTOR_DB:
    similarity = cosine_similarity(query_embedding, embedding)
    similarities.append((chunk, similarity))
  similarities.sort(key=lambda x: x[1], reverse=True)
  return similarities[:top_n]

Строим промпт

In [8]:
# input_query = 'tell me about cat speed, задавали в самом начале
retrieved_knowledge = retrieve(input_query)

print('Retrieved knowledge:')
for chunk, similarity in retrieved_knowledge:
  print(f' - (similarity: {similarity:.2f}) {chunk}')

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {chunk}' for chunk, similarity in retrieved_knowledge])}
'''

Retrieved knowledge:
 - (similarity: 0.78) A cat can travel at a top speed of approximately 31 mph (49 km) over a short distance.

 - (similarity: 0.67) Cats sleep 16 to 18 hours per day. When cats are asleep, they are still alert to incoming stimuli. If you poke the tail of a sleeping cat, it will respond accordingly.

 - (similarity: 0.67) Researchers are unsure exactly how a cat purrs. Most veterinarians believe that a cat purrs by vibrating vocal folds deep in the throat. To do this, a muscle in the larynx opens and closes the air passage about 25 times per second.



In [9]:
stream = ollama.chat(
  model=LANGUAGE_MODEL,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Chatbot response:
Cats are known for their agility and quick movements on land. According to the given context, a cat can travel at approximately 31 mph (49 km) over a short distance. This makes them one of the fastest animals on four legs!